In [ ]:
import os, sys, importlib
import rasterio

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import Point
from shapely.wkt import loads
from rasterio import features

sys.path.append("../")
import infrasap.vulnerability_mapping as vulmap
import infrasap.GOSTRocks.rasterMisc as rMisc
import infrasap.GOSTRocks.misc as misc


# Calculate Vulnerability

Based on established CoVID hospitalization rates, we combine demographic data to map population at risk.

In [ ]:
# https://mrc-ide.github.io/global-lmic-reports/parameters.html
# https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm
vul_def = {'0-5'  :0.001,
           '6-10' :0.001,
           '11-15':0.001,
           '16-20':0.002,
           '21-25':0.005,
           '26-30':0.010,
           '31-35':0.016,
           '36-40':0.023,
           '41-45':0.029,
           '46-50':0.039,
           '51-55':0.058,
           '56-60':0.072,
           '61-65':0.102,
           '66-70':0.117,
           '71-75':0.146,
           '76-80':0.177,
           '81-100':0.180}

pop_folder = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/GLOBAL_1km_Demographics"
pop_files = os.listdir(pop_folder)
in_admin = '/home/wb411133/data/Projects/CoVID/IDN/adm0.shp'
out_folder = '/home/wb411133/data/Projects/CoVID/IDN/wp_files'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [ ]:
wp_files = []
#Extract country specific pop files
for pFile in pop_files:
    curR = rasterio.open((os.path.join(pop_folder, pFile)))
    out_file = os.path.join(country_folder, pFile)
    if not os.path.exists(out_file):
        rMisc.clipRaster(curR, country_bounds, out_file)
    wp_files.append(out_file)

In [ ]:
#Calculate vulnerability
wp_file_objects  = [vulmap.wp_demographics(os.path.join(country_folder, x)) for x in wp_files]
vul = vulmap.wp_vulnerability(wp_file_objects, vul_def)
vul.calculate_vulnerability()
vul.combine_results(out_vulnerability, '')
for f in wp_files:
    os.remove(f)